<a href="https://colab.research.google.com/github/gerasimos-matidis/my_icdar/blob/main/segm_w_on_the_fly_augm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Segmentation with on the fly Data Augmentation


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from gm_networks import unet_model
from utils import center_crop

In [ ]:
# Load the initial input and output images and expand their dimensions to fit the ImageDataGenerator 
# For inputs: initial_dimensions = (width, height, 3) ---> new_dimensions = (1, width, height, 3)
# For outputs: initial_dimensions = (width, height) ---> new_dimensions = (1, width, height, 1)
x_initial_train = plt.imread('train/101-INPUT.jpg')
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')
x_initial_train = np.expand_dims(x_initial_train, 0) 
y_initial_train = np.expand_dims(y_initial_train, (0, -1)) 

In [ ]:
# Model/training hyperparameters
INPUT_SHAPE = (512, 512, 3)
INITIAL_FILTERS = 32
PADDING = 'same'

TRAIN_IMAGES_NUMBER = 500

BATCH_SIZE = 8
EPOCHS = 800
steps_per_epoch = TRAIN_IMAGES_NUMBER // BATCH_SIZE# Define the new size to create images of size (NEW_SIZE, NEW_SIZE), as well as define the number of train and validation images to be created

In [ ]:
model = unet_model(input_shape=INPUT_SHAPE, initial_filters=INITIAL_FILTERS, padding=PADDING)


In [ ]:
print(model.output_shape[1:3])
print(y_initial_train.shape)
# TODO: center_crop the y_initial_train tensor 

In [ ]:
new_size = INPUT_SHAPE[0]

# Concatenate the respective inputs and outputs to ensure that they will undergo the same crop
concatenated_images_train = np.concatenate([x_initial_train, y_initial_train], -1)

# Initialize the tensors for the training batches
croped_images_train = np.zeros([TRAIN_IMAGES_NUMBER, new_size, new_size, 4])

# Random croping
for i in range(TRAIN_IMAGES_NUMBER):
    
    t = tf.image.random_crop(concatenated_images_train, size=[1, new_size, new_size, 4])
    croped_images_train[i] = t[0]

# Separate the inputs from the outputs to create the final batches
x_train = croped_images_train[:, :, :, :3]
y_train = np.expand_dims(croped_images_train[:, :, :, 3], -1)

if PADDING == 'valid':
    y_train = center_crop(y_train, model.output_shape[1:3])


In [ ]:
print(x_train.shape)

In [ ]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [ ]:
# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [ ]:
# Define the model and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS)

In [ ]:
import os
# save the model and history of the training (loss, accuracy)
model_path = f'models/Unet_inputsize{new_size}_initialfilters{INITIAL_FILTERS}_padding_{PADDING}_epochs{EPOCHS}' #TODO: loss and metric'
model.save(model_path)


In [ ]:
np.save(os.path.join(model_path, 'history'), history.history)